# 6-3 推薦系統
參考範例：https://github.com/QiuBingCheng/Recommendation-System/blob/master/recommendation.py 。

請使用anime.csv結合mapping_id.csv資料，幫助你愛看動漫的室友找到擁有相似喜好的朋友。（請先篩選mapping_id檔案當中前100萬份資料進行分析，並將Space的分類排除。）

#### 請問以下哪個選項並非 user_id=428 喜歡的前十位動漫名單，請依照anime_id回答。
33581 - Trinity Seven Movie: Eternity Library to Alchemic Girl 

#### 依照動漫名稱去尋找相似的動漫迷們，試問anime_id = 15583的前五位user_id回傳結果
4594, 9172, 8465, 8745, 9356 

請你嘗試驗證看看，兩位使用者是否擁有相同喜好的動漫，請以anime_id作為回答。

#### 請問 user=8892 跟 user=3392 共同喜好為以下何者？請問何者非 user=428 跟 user=1276 的共同喜好？

(20583, 30276) 

#### 請幫user=1推薦與他相似的人還看了哪些動漫吧，請找出前10部分數最高的人

## 6-3-1: 基於內容過濾 (Content-based Filtering)

### 1. 載入資料與前處理

In [2]:
movie = pd.read_csv('dataset/movie_lens/movie.csv')
rating = pd.read_csv('dataset/movie_lens/rating.csv')
rating.drop(['timestamp'], axis=1, inplace=True)
df = pd.merge(rating, movie, on='movieId')
df_cb = df.loc[:,['userId','movieId','genres']]
df_cb

,userId,movieId,genres
0,1,1,Adventure|Animation|Children|Comedy|Fantasy
1,5,1,Adventure|Animation|Children|Comedy|Fantasy
2,7,1,Adventure|Animation|Children|Comedy|Fantasy
3,15,1,Adventure|Animation|Children|Comedy|Fantasy
4,17,1,Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...
100831,610,160341,Action|Thriller
100832,610,160527,Action|Crime|Drama
100833,610,160836,Action|Drama|Thriller
100834,610,163937,Horror|Thriller


#### 製作電影向量 (item vector)

In [3]:
one_hot = movie["genres"].str.get_dummies('|')
movie_vec = pd.concat([movie, one_hot], axis=1)
movie_vec.drop(['genres','title'], axis=1, inplace=True)
movie_vec.set_index("movieId", inplace=True)
movie_vec.head(5)

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### 製作用戶向量 (user vector)

In [4]:
one_hot = df["genres"].str.get_dummies('|')
user_vec = pd.concat([df, one_hot], axis=1)
user_vec.drop(['movieId', 'genres', 'rating'], axis=1, inplace=True)
user_vec = user_vec.groupby("userId").mean()
user_vec

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
userId,,,,,,,,,,,,,,,,,,,,
1,0.0,0.387931,0.366379,0.125000,0.181034,0.357759,0.193966,0.000000,0.293103,0.202586,0.004310,0.073276,0.000000,0.094828,0.077586,0.112069,0.172414,0.237069,0.094828,0.030172
2,0.0,0.379310,0.103448,0.000000,0.000000,0.241379,0.344828,0.103448,0.586207,0.000000,0.000000,0.034483,0.137931,0.000000,0.068966,0.034483,0.137931,0.344828,0.034483,0.034483
3,0.0,0.358974,0.282051,0.102564,0.128205,0.230769,0.051282,0.000000,0.410256,0.102564,0.000000,0.205128,0.000000,0.025641,0.025641,0.128205,0.384615,0.179487,0.128205,0.000000
4,0.0,0.115741,0.134259,0.027778,0.046296,0.481481,0.125000,0.009259,0.555556,0.087963,0.018519,0.018519,0.004630,0.074074,0.106481,0.268519,0.055556,0.175926,0.032407,0.046296
5,0.0,0.204545,0.181818,0.136364,0.204545,0.340909,0.272727,0.000000,0.568182,0.159091,0.000000,0.022727,0.068182,0.113636,0.022727,0.250000,0.045455,0.204545,0.068182,0.045455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.135426,0.131839,0.037668,0.043946,0.377578,0.119283,0.004484,0.626009,0.086996,0.007175,0.046637,0.014350,0.039462,0.081614,0.318386,0.070852,0.178475,0.058296,0.015247
607,0.0,0.385027,0.240642,0.032086,0.101604,0.294118,0.144385,0.000000,0.438503,0.112299,0.000000,0.187166,0.005348,0.026738,0.090909,0.155080,0.192513,0.326203,0.032086,0.010695
608,0.0,0.333333,0.217810,0.066185,0.105897,0.427196,0.175692,0.007220,0.336943,0.133574,0.004813,0.116727,0.014440,0.039711,0.083032,0.127557,0.200963,0.311673,0.022864,0.013237


### 2. 計算距離(相似度)

#### 製作用戶電影矩陣 (user x movie matrix)

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

user_movie = cosine_similarity(user_vec.values, movie_vec.values)
user_movie = 1 - user_movie
user_movie = pd.DataFrame(user_movie, index=user_vec.index, columns=movie_vec.index)
user_movie

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,0.365298,0.501486,0.617527,0.492891,0.588124,0.455645,0.617527,0.554367,0.553387,0.341045,...,0.399554,0.659634,0.470153,0.856091,1.000000,0.382185,0.544461,0.662559,0.582438,0.588124
2,0.832985,0.935315,0.788741,0.460961,0.738580,0.331592,0.788741,0.920778,0.589197,0.482523,...,0.589197,0.551074,0.366223,1.000000,0.887963,0.663889,0.849069,0.365123,0.709519,0.738580
3,0.556036,0.652633,0.702195,0.478950,0.729254,0.600528,0.702195,0.659651,0.578840,0.444213,...,0.368260,0.574564,0.468205,0.879669,1.000000,0.533716,0.704738,0.518675,0.617108,0.729254
4,0.591524,0.817942,0.377209,0.114821,0.434574,0.717496,0.377209,0.850069,0.864080,0.711218,...,0.600396,0.515607,0.138856,0.967379,0.989126,0.581367,0.595078,0.347585,0.880824,0.434574
5,0.490475,0.649177,0.534525,0.254501,0.620223,0.561471,0.534525,0.695651,0.772134,0.619942,...,0.594904,0.445011,0.283885,0.848089,1.000000,0.531608,0.590706,0.367038,0.731457,0.620223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.649369,0.824563,0.430937,0.117428,0.563388,0.710799,0.430937,0.856268,0.843401,0.702416,...,0.640651,0.457337,0.179406,0.956443,0.994815,0.631317,0.664694,0.276116,0.858468,0.563388
607,0.594780,0.695434,0.631373,0.405201,0.658662,0.426700,0.631373,0.719142,0.553157,0.362204,...,0.475580,0.613820,0.398787,0.962763,1.000000,0.522126,0.706184,0.491095,0.657704,0.658662
608,0.490303,0.683486,0.529721,0.382798,0.487849,0.431941,0.529721,0.725586,0.600378,0.402789,...,0.383976,0.658257,0.352219,0.920653,0.991344,0.424371,0.566043,0.596050,0.661318,0.487849


### 3. 進行推薦

#### 尋找與指定用戶距離最近(最相似)的前幾部電影

In [6]:
def similar_movies(user_id, user_movie_matrix, num):
    user_vec = user_movie_matrix.loc[user_id].values 
    sorted_index = np.argsort(user_vec)[:num] # 排序前num名
    return list(user_movie_matrix.columns[sorted_index])

#### 尋找與指定電影距離最近(最相似)的前幾個用戶

In [7]:
def similar_users(movie_id, user_movie_matrix, num):
    movie_vec = user_movie_matrix[movie_id].values 
    sorted_index = np.argsort(movie_vec)[:num] # 排序前num名
    return list(user_movie_matrix.index[sorted_index])    

對用戶1推薦10部影片：

In [8]:
similar_movies(user_id=1, user_movie_matrix=user_movie, num=10) # 輸入用戶的ID，返回最相似的影片名單

[117646, 55116, 4956, 6990, 5657, 164226, 4818, 6503, 80219, 26184]

對電影1推薦10個用戶：

In [9]:
similar_users(movie_id=1, user_movie_matrix=user_movie, num=10) # 輸入影片的ID，返回最相似的用戶名單

[401, 388, 252, 161, 20, 539, 306, 276, 234, 605]

## 6-3-2: 協同式過濾 (Collaborative Filtering)

### 1. 讀取資料與前處理

In [10]:
df_cf = df.loc[:,['userId','movieId','rating','title']]
df_cf

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)
...,...,...,...,...
100831,610,160341,2.5,Bloodmoon (1997)
100832,610,160527,4.5,Sympathy for the Underdog (1971)
100833,610,160836,3.0,Hazard (2005)
100834,610,163937,3.5,Blair Witch (2016)


### 2. 計算距離(相似度)

由於[sklearn的cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)的輸入和輸出都是一個「矩陣」與我們想要的不同；<br>
我們想要的是輸入兩個向量(x,y)可以返還一個「值」(x和y的cosine_similarity)。<br>
因此我們要自己寫一個cosine_similarity的計算函式：


In [11]:
def my_cosine_similarity(vec1, vec2): # 計算兩個向量之間的餘弦相似性
    vec1 = np.mat(vec1) # 向量a
    vec2 = np.mat(vec2) # 向量b
    num = float(vec1 * vec2.T)
    denom = np.linalg.norm(vec1) * np.linalg.norm(vec2)
    if denom == 0:
        cos = 0.0
    else:
        cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim

#### 找出兩人共同觀看過的電影

In [12]:
def common_movie(user1, user2):
    s1 = set((df_cf.loc[df_cf["userId"]==user1, "movieId"].values))
    s2 = set((df_cf.loc[df_cf["userId"]==user2, "movieId"].values))
    return s1.intersection(s2)

#### 計算兩人對於特定電影評分的相似度

In [13]:
def user_rating_similarity(user1, user2, movies_id):
    u1 = df_cf[df_cf["userId"]==user1]
    u2 = df_cf[df_cf["userId"]==user2]
    vec1 = u1[u1.movieId.isin(movies_id)].sort_values(by="movieId")["rating"].values
    vec2 = u2[u2.movieId.isin(movies_id)].sort_values(by="movieId")["rating"].values
    return my_cosine_similarity(vec1, vec2)

### 3. 進行推薦

In [14]:
from collections import defaultdict

def recommend(user, num=10):

    # 計算該用戶與其他用戶的相似度
    user_similarity = []
    other_user_list = []
    for other_user in df_cf.userId.unique():
        if other_user == user: # 跳過自己
            continue
        other_user_list.append(other_user) # 紀錄其他用戶ID
        common_movies = common_movie(user, other_user) # 找出兩人共同看過的電影
        sim = user_rating_similarity(user, other_user, common_movies) # 計算兩人評分的相似性
        user_similarity.append([other_user, sim])

    # 排序前10個相似的用戶
    user_similarity = np.array(user_similarity)
    sorted_index = np.argsort(user_similarity, axis=0)[:,1][::-1][:10]
    top10_similar_user = user_similarity[:,0][sorted_index]
    
    # 找出哪些電影是該用戶還沒看過的
    seen_movies = df_cf.loc[df_cf["userId"]==user,"movieId"].values # 看過的電影
    not_seen_movies = defaultdict(list) # 未看過的電影
    for similar_user in top10_similar_user: 
        movies = df_cf.loc[df_cf.userId==similar_user, ["movieId","rating"]].values.tolist() # 找出前10個相似的其他用戶看過的電影及評分
        if isinstance(movies[0], list):
            for movie in movies:
                if movie[0] in seen_movies: # 若自己也看過該電影就跳過
                    continue
                not_seen_movies[movie[0]].append(movie[1]) # 找出自己沒看過，但其他人看過的電影
        elif movies[0] not in seen_movies:
            not_seen_movies[movies[0]].append(movies[1])
                
    # 將其他用戶對同一部電影的評分取平均值
    for movie in not_seen_movies:
        not_seen_movies[movie] = np.mean(not_seen_movies[movie])
    
    # 排序ratingy在前10名的電影
    top10_rating = sorted(not_seen_movies.items(), key=lambda x: x[1], reverse=True)
    return [movie for movie,rating in top10_rating][:num], other_user_list

In [15]:
top10_movie, other_user_list = recommend(user=1, num=10)

In [16]:
print("其他相似的用戶：", other_user_list)

其他相似的用戶： [5, 7, 15, 17, 18, 19, 21, 27, 31, 32, 33, 40, 43, 44, 45, 46, 50, 54, 57, 63, 64, 66, 68, 71, 73, 76, 78, 82, 86, 89, 90, 91, 93, 96, 98, 103, 107, 112, 119, 121, 124, 130, 132, 134, 135, 137, 140, 141, 144, 145, 151, 153, 155, 156, 159, 160, 161, 166, 167, 169, 171, 177, 178, 179, 182, 185, 186, 191, 193, 200, 201, 202, 206, 213, 214, 216, 217, 219, 220, 223, 226, 229, 232, 233, 234, 239, 240, 247, 249, 252, 254, 263, 264, 266, 269, 270, 273, 274, 275, 276, 277, 279, 280, 282, 283, 288, 290, 291, 292, 293, 298, 304, 307, 314, 322, 323, 328, 330, 332, 334, 336, 337, 339, 341, 347, 350, 353, 357, 359, 364, 367, 372, 373, 378, 380, 381, 382, 385, 389, 391, 396, 399, 401, 411, 412, 414, 420, 422, 432, 434, 436, 438, 443, 448, 451, 453, 456, 460, 462, 468, 469, 470, 471, 474, 476, 477, 480, 483, 484, 488, 490, 492, 500, 504, 509, 514, 517, 522, 524, 525, 528, 529, 533, 534, 541, 544, 550, 555, 559, 560, 561, 562, 567, 570, 572, 573, 579, 580, 584, 587, 590, 596, 597, 599, 600, 60

In [17]:
print(f'前10部電影推薦名單： {["{:.0f}".format(x) for x in top10_movie]}')


前10部電影推薦名單： ['1641', '1719', '314', '1357', '1207', '1237', '1633', '1643', '1185', '53']
